In [1]:
# !pip3 install promise tensorflow-datasets dill future tensorflow-metadata
# !pip3 install tensorflow-datasets tf-sentencepiece sentencepiece tensorflow-text==1.15 tfds-nightly --no-deps
# !pip3 install t5[gcp]

In [2]:
# !pip3 install tensorflow==1.15 tensorflow-text==1.15

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
t5.__version__

'0.5.0'

In [4]:
# tf.compat.v1.disable_v2_behavior()
# tf.compat.v1.enable_eager_execution()

In [5]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver('node-1', 'us-central1-a')
TPU_ADDRESS = tpu.get_master()
TPU_ADDRESS

'grpc://10.88.120.26:8470'

In [6]:
vocab = 'gs://mesolitica-general/t5-vocab/sp10m.cased.t5.model'

In [7]:
def dumping_dataset(split, shuffle_files=False):
    del shuffle_files
    ds = tf.data.TextLineDataset(['gs://mesolitica-general/t5-data/dumping-iium.tsv'])

    ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.map(lambda *ex: dict(zip(["title", "text"], ex)))
    return ds

t5.data.TaskRegistry.remove('dumping_txt')
t5.data.TaskRegistry.add(
    "dumping_txt",
    dataset_fn=dumping_dataset,
    splits=["train"],
    text_preprocessor=functools.partial(
        t5.data.preprocessors.rekey, key_map={"inputs": None, "targets": "text"}),
    token_preprocessor=t5.data.preprocessors.unsupervised,
    sentencepiece_model_path=vocab,
    metric_fns=[])

In [8]:
# nq_task = t5.data.TaskRegistry.get("dumping_txt")
# ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 1024, "targets": 32})
# for ex in tfds.as_numpy(ds.take(5)):
#     print(ex)

In [9]:
t5.data.MixtureRegistry.remove("trivia_all")
t5.data.MixtureRegistry.add(
    "trivia_all",
    ['dumping_txt'],
     default_rate=1.0
)

In [10]:
import gin

gin.parse_config_file('pretrained_models_base_operative_config.gin')

In [11]:
import mesh_tensorflow
mesh_tensorflow.transformer.utils.tpu_mesh_shape()

Shape[batch=8]

In [12]:
model = t5.models.MtfModel(
    model_dir='gs://mesolitica-general/t5-base/',
    tpu=TPU_ADDRESS,
    batch_size=128,
    sequence_length={"inputs": 1024, "targets": 1024},
#     learning_rate_schedule=0.003,
#     save_checkpoints_steps=5000,
#     iterations_per_loop=100,
)

In [13]:
model.train(mixture_or_task_name='trivia_all', steps=100)

INFO:tensorflow:Using config: {'_model_dir': 'gs://mesolitica-general/t5-base/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.88.120.26:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f677f1d4c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.88.120.26:8470', '_evaluation_master': 'grpc://10.88.120.26:8470', '_is_chief': True, '_num_ps_replica

ValueError: Tensor dimension size not divisible by mesh dimension size: tensor_shape=Shape[heads=12, buckets=32] tensor_layout=TensorLayout(0, None)